## 요구사항2
### titanic 데이터에 맞게 수정한 코드

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
# 루트 경로 설정!
from pathlib import Path
BASE_PATH = str(Path(" ").resolve().parent.parent.parent) # BASE_PATH: /Users/yhhan/git/link_dl
# 루트 경로 추가!
import sys
sys.path.append(BASE_PATH)

In [2]:
# import Titanic dataset!
from _02_homeworks._02_fcn_dl.titanic.titanic_dataset\
  import get_preprocessed_dataset

In [3]:
# dataset 가져오기!
def get_data():
# 클래스 변수 선언!
  #titanic_dataset = get_preprocessed_dataset()
  #print(titanic_dataset)

# 데이터 불러오기
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(len(train_dataset), len(validation_dataset), len(test_dataset))
# 데이터를 배치 사이즈로 셔플해서 불러오기
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
# 셔플한 데이터 호출
  return train_data_loader, validation_data_loader, test_data_loader

In [4]:
# 내 모델(요구사항 2 변경x)
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

In [5]:
# 모델 가져와서 최적화하기
def get_model_and_optimizer():
  # 나의 모델의 입력값가 출력값을 11과 1로 설정
  my_model = MyModel(n_input=9, n_output=1)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

In [6]:
# 모델 반복
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  # 전체 학습 = 1회 ==> 횟수
  n_epochs = 1000
  # 함수를 객체화
  loss_fn = nn.MSELoss()  # Use a built-in loss function
  # 100마다 출력
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    # 훈련 손실 및 데이터 개수 초기화
    loss_train = 0.0
    num_trains = 0
    # 미니배치 가져와서
    for train_batch in train_data_loader:
      # 예측값 계산
      # input_size = train_batch['input'].shape
      # target_size = train_batch['target'].shape
      #
      # print(f"Input size: {input_size}, Target size: {target_size}")
      output_train = model(train_batch['input'])
      target_train = train_batch['target'].unsqueeze(1).float()  # 형태를 [10]에서 [10, 1]로 변경
      loss = loss_fn(output_train, target_train)
      #print(loss)
      # output_train = model(train_batch['input'])
      # print(output_train)
      # # 손실 확인
      #loss = loss_fn(output_train, train_batch['target'])
      # # 미니 배치 손실 누적
      loss_train += loss.item()
      # 훈련 횟수 증가
      num_trains += 1

      # 경사하강법 -> 모델 파라미터 업데이트
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    # 그래디언트 비활성화
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        target_validation = validation_batch['target'].unsqueeze(1).float()  # 형태를 [10]에서 [10, 1]로 변경
        loss = loss_fn(output_validation, target_validation)

        # output_validation = model(validation_batch['input'])
        # loss = loss_fn(output_validation, validation_batch['target'])
        loss_validation += loss.item()
        num_validations += 1

    # 기록
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

In [7]:
# 테스트 코드 추가하기
def test(test_data_loader):
  import pandas as pd
  print("[TEST]")
  batch = next(iter(test_data_loader))
  #print("!!!!!!!!")
  #print(batch)
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=9, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)

  results = []
  output_path = 'predictions.csv'
  for idx, prediction in enumerate(prediction_batch, start=892):
    results.append([idx, prediction.item()])

    # 결과를 데이터프레임으로 변환
  df = pd.DataFrame(results, columns=['Index', 'Prediction'])

    # CSV 파일로 저장
  df.to_csv(output_path, index=False)
  # 예측 결과를 저장할 파일 경로

In [8]:
def main(args):
  # 현재 시간 출력
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
  # 몇번 학습했는지
  # 배치 사이즈는 얼마인지
  # 학습률은 얼마인지
  # 숨겨진 유닛의 개수는 몇개인지
  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
  }
  # wnadb 사용
  wandb.init(
    mode="online",#if args.wandb else "disabled",
    project="titanic_test",
    notes="titanic_test",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  #
  #print("!!")
  print(args)
  print(wandb.config)
  #print("!!")
  # 배치로 셔플한 데이터 가져오기
  train_data_loader, validation_data_loader, test_data_loader = get_data()
  # 모델 최적화 및 가져오기 입력 11개 출력 1개로 변경
  linear_model, optimizer = get_model_and_optimizer()
  #wandb로 출력하기
  wandb.watch(linear_model)

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )

  test(test_data_loader)

  wandb.finish()

In [9]:
# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args()

  main(args)

usage: ipykernel_launcher.py [-h] [--wandb | --no-wandb] [-b BATCH_SIZE] [-e EPOCHS]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\태경\AppData\Roaming\jupyter\runtime\kernel-e465a587-2871-47d1-a197-a00723e90625.json


SystemExit: 2

Z:\Program Files\anaconda3\envs\link_dl\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Wandb로 그래프 얻기

https://wandb.ai/taeya/titanic_test?workspace=user-taeya

### 모델 구성내에 더 나은 성능을 산출하는 함수 찾기

nn.ELU()와 nn.ReLU() 를 같이 사용하는 코드가 좀 더 나은 성능을 산출하였음.

## 요구사항 3

훈련 과정 중 어느 에포크 시점에 테스트를 수행해야하는가?

테스트를 수행함에 있어 적절한 에포크 타임은 직접 시도하면서 너무 많지도 너무 적지도 않은 순간에 csv를 산출해야한다.

너무 적거나 많은 에포크는 오히려 성능을 줄이는 원인이 될 수 있다.

## 요구사항 4

https://drive.google.com/file/d/18IxbTF-GBAgB_A0gC36lJM4Fyy2Mlqd9/view?usp=sharing

사진 넣는법을 모르겠기에 이렇게 넣습니다..

wandb url 제출

https://wandb.ai/taeya/titanic_test/runs/w0nnphly?workspace=user-taeya

숙제 후기

딥러닝 과제를 수행하면서 교수님이 주신 코드를 해석하는 과정에서 딥러닝에 대해서 이해가 안되던 부분들에 대해 다시 상기하면 공부할 수 있는 기회를 얻었고 이를통해서 그저 수업을 들었을 때와 다르게 직접 시험하면서 실력을 향상을 할 수 있는 계기가 되었다.
특히 코드를 다른 데이터로 수정할 때 어려움이 있었으나 시간을 투자하여 이를 해결할 수 있었다.